In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml

Prepare data

In [ ]:
data = fetch_openml(name='adult', version=2, as_frame='auto')
df = data.frame

ref = df[:20000]
curr = df[20000:]

curr['target'] = curr['education-num']
curr['preds'] = curr['education-num'].values + np.random.normal(0, 6, curr.shape[0])
ref['target'] = ref['education-num']
ref['preds'] = ref['education-num'].values + np.random.normal(0, 6, ref.shape[0])

curr.iloc[:2000, 3:5] = np.nan
curr.iloc[:2000, 12] = np.nan

Create and run a report with separate metrics and metric presets

In [ ]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDrift
from evidently.metrics import ClassificationPerformanceMetrics
from evidently.metrics import DataQualityMetrics

report = Report(metrics=[
#     DataIntegrityMetrics(),
    DataDrift(),
#     DataQualityMetrics()
])

report.run(reference_data=ref, current_data=curr, column_mapping=ColumnMapping(target='target', prediction='preds'))
report

In [ ]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDrift
from evidently.metrics import DataIntegrityMetrics
from evidently.metrics import DataQualityMetrics

report = Report(metrics=[
#     DataIntegrityMetrics(),
    DataDrift(),
#     DataQualityMetrics()
])

report.run(reference_data=ref, current_data=curr, column_mapping=ColumnMapping(target='target', prediction='preds'))
report

Binary classification metric preset

In [1]:
import pandas as pd

from datetime import datetime
from sklearn import datasets
from sklearn import ensemble
from sklearn import model_selection

from evidently.pipeline.column_mapping import ColumnMapping


bcancer = datasets.load_breast_cancer()
bcancer_frame = pd.DataFrame(bcancer.data, columns = bcancer.feature_names)
bcancer_frame['target'] = bcancer.target
target = 'target'
prediction = 'prediction'

numerical_features = bcancer.feature_names
categorical_features = []

features = numerical_features.tolist() + categorical_features
train_data, test_data = model_selection.train_test_split(bcancer_frame, random_state=0)
model = ensemble.RandomForestClassifier(random_state=0)
model.fit(train_data[features], train_data.target)
train_predictions = model.predict(train_data[features])
test_predictions = model.predict(test_data[features])
train_data['prediction'] = train_predictions
test_data['prediction'] = test_predictions

bcancer_column_mapping = ColumnMapping()
bcancer_column_mapping.target = target
bcancer_column_mapping.prediction = prediction
bcancer_column_mapping.numerical_features = numerical_features

/Users/tapot/opt/anaconda3/envs/evidently_dev_main/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/tapot/opt/anaconda3/envs/evidently_dev_main/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [2]:
from evidently.report import Report
from evidently.metrics import ClassificationPerformanceMetrics

report = Report(metrics=[ClassificationPerformanceMetrics()])
report.run(current_data=test_data, reference_data=train_data, column_mapping=bcancer_column_mapping)
report